<a href="https://colab.research.google.com/github/nanopiero/fusion/blob/main/notebooks/fcns/training_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A2.radar + cmls -> rain gauges 1 min + rain gauges 60 min [xrl_yg1g60]

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import sys
sys.path.append('/home/mdso/lepetitp/ppc/WEBCAMS/src/raincell/ia/notebooks/learning/simulation')

from fusion.utils.datasets import spatialized_gt, create_cmls_filter, FusionDataset
from fusion.utils.datasets import indices_to_sampled_values, get_point_measurements, point_gt, segment_gt, make_noisy_images
from torch.utils.data import DataLoader
from fusion.utils.fcn import UNet
from fusion.utils.cost_functions import QPELoss_fcn, compute_metrics
from fusion.utils.viz import set_tensor_values2, plot_images, plot_images_10pts_20seg, plot_results_10pts_20seg

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# config de base (change en B.):
num_epochs = 1500
save_every = 10
path = r'/scratch/mdso/lepetitp/ppc/RAINCELL/models/simulation/checkpoint_fcn_exp_A2_xrl_yg1g60.pt'
npoints = 10
npairs = 20
nsteps = 60
ndiscs = 5
size_image = 64
length_dataset = 6400
device = torch.device('cuda:0')

In [4]:
# Dataset, DataLoader
dataset = FusionDataset(length_dataset=length_dataset,
                        npairs=npairs,
                        nsteps=nsteps,
                        ndiscs=ndiscs, size_image=size_image)


loader = DataLoader(dataset, batch_size=64, num_workers=8)

In [5]:
# Petit UNet
ch_in = 72
ch_out = nsteps * 3 + 1
size = nsteps * 3
device = torch.device('cuda:0')
model = UNet(ch_in, ch_out, size, nb_additional_parameters=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [6]:
criterion = QPELoss_fcn(eval_qpe_1h=True)
# Baseline with a FCN
use_fcn = True

best_loss = [float('inf'), float('inf')]  # Initialize best validation loss to a very high value
losses = []
last_epoch = 0

In [7]:
checkpoint = torch.load(path, \
                            map_location=device)
last_epoch = checkpoint['epoch']
losses = checkpoint['train_losses']
# best_loss = checkpoint['best_loss']
model_weights = checkpoint['model']
optimizer_state_dict = checkpoint['optimizer']
# scheduler_state_dict = checkpoint['scheduler']
model.load_state_dict(model_weights)
optimizer.load_state_dict(optimizer_state_dict)
# scheduler.load_state_dict(scheduler_state_dict)
del checkpoint, model_weights, optimizer_state_dict

In [8]:
last_epoch

1000

In [9]:
model.train()
for epoch in range(last_epoch, num_epochs + 1):
  print('epoch n°', epoch)
  running_regression_loss = 0.0
  running_regression_loss_1h = 0.0
  running_segmentation_loss = 0.0
  train_confusion_matrix = np.zeros((2, 2), dtype=int)
  for i, (images, pairs, filters) in enumerate(loader):

    # ground truth (not usable)
    images = images.clone().detach().float().to(device)

    # pseudo CMLs
    pairs = pairs.clone().detach().float().to(device)
    filters = filters.clone().float().detach().to(device)

    # for transformers :
    # segment_measurements = segment_gt(images, pairs, filters)
    _, segment_measurements_fcn = segment_gt(images, pairs, filters,
                                             use_fcn=use_fcn)

    # pseudo pluvios
    _, point_measurements_fcn, _ = point_gt(images, npoints=npoints,
                                            use_fcn=use_fcn)


    # pseudo radar
    noisy_images = make_noisy_images(images)

    # prepare inputs and targets
    inputs = torch.cat([noisy_images, segment_measurements_fcn], dim=1)
    targets = point_measurements_fcn


    optimizer.zero_grad()  # Zero the gradients
    outputs = model(inputs)  # Forward pass

    regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, _  = criterion(model.p, outputs, targets)
    loss.backward()  # Backward pass
    optimizer.step()  # Update the weights

    del inputs, targets, outputs, loss, noisy_images, images, pairs, filters
    torch.cuda.empty_cache()

    running_regression_loss += regression_loss
    running_regression_loss_1h += regression_loss_1h
    running_segmentation_loss += segmentation_loss
    train_confusion_matrix += batch_cm

  # Calculating average training loss
  train_regression_loss = running_regression_loss / len(loader)
  train_regression_loss_1h = running_regression_loss_1h / len(loader)
  train_segmentation_loss = running_segmentation_loss / len(loader)
  losses.append((epoch, train_regression_loss, train_regression_loss_1h, train_segmentation_loss, train_confusion_matrix))
  print(f'Training, Regression Loss: {train_regression_loss:.4f}, Regression Loss 1h: {train_regression_loss_1h:.4f}, Segmentation Loss:{train_segmentation_loss:.4f}' )
  print("Train Confusion Matrix:")
  print(train_confusion_matrix)
  accuracy, csi, sensitivity, specificity, false_alarm_ratio = compute_metrics(train_confusion_matrix)
  print(f'Accuracy: {accuracy:.4f}, CSI: {csi:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}, False Alarm Ratio: {false_alarm_ratio:.4f}')
  print('\n')
    
  if (epoch % save_every == 0 or \
    epoch == last_epoch):
    print("saving step")
    checkpoint = { 
        'epoch': epoch,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        # 'scheduler': scheduler.state_dict(),
        'train_losses': losses,
        }
    torch.save(checkpoint, path)  


epoch n° 1000
Training, Regression Loss: 0.0423, Regression Loss 1h: 1.5659, Segmentation Loss:0.1319
Train Confusion Matrix:
[[3129046   74795]
 [ 119658  516501]]
Accuracy: 0.9494, CSI: 0.7265, Sensitivity: 0.8119, Specificity: 0.9767, False Alarm Ratio: 0.1265


saving step
epoch n° 1001
Training, Regression Loss: 0.0419, Regression Loss 1h: 1.5935, Segmentation Loss:0.1369
Train Confusion Matrix:
[[3126231   76796]
 [ 125246  511727]]
Accuracy: 0.9474, CSI: 0.7169, Sensitivity: 0.8034, Specificity: 0.9760, False Alarm Ratio: 0.1305


epoch n° 1002
Training, Regression Loss: 0.0427, Regression Loss 1h: 1.7059, Segmentation Loss:0.1313
Train Confusion Matrix:
[[3118807   75794]
 [ 117960  527439]]
Accuracy: 0.9495, CSI: 0.7313, Sensitivity: 0.8172, Specificity: 0.9763, False Alarm Ratio: 0.1256


epoch n° 1003
Training, Regression Loss: 0.0429, Regression Loss 1h: 1.6512, Segmentation Loss:0.1313
Train Confusion Matrix:
[[3124728   76006]
 [ 118381  520885]]
Accuracy: 0.9494, CSI: 0.

KeyboardInterrupt: 